# TODO: Make simpler

In [1]:
%cd ../src/

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/src


In [2]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import KFold

from concepts import concept_instances
from utils import load_data, load_q_network_device, prepare_folders

In [ ]:
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [3]:
max_data = 1000
data = load_data()
for concept in concept_instances.values():
    concept.prepare_data(data, max_size=max_data)

In [4]:
q_network, device = load_q_network_device()

In [6]:
def binary_accuracy(preds, y):
    preds = preds > 0.5
    correct = (preds == y).float()
    acc = correct.sum() / len(correct)
    return max(0, 2*acc-1)

def r2(preds, y):
    return max(0, r2_score(y.detach().cpu().numpy(), preds.detach().cpu().numpy()))

def prepare_data(concept):
    obs_train = torch.tensor(concept.obs_train).float().to(device)
    obs_test = torch.tensor(concept.obs_test).float().to(device)
    _, acts_dict_train = q_network(obs_train, return_acts=True)
    _, acts_dict_test = q_network(obs_test, return_acts=True)
    acts_dict_train_test = {}
    for k in acts_dict_train.keys():
        acts_dict_train_test[k] = (acts_dict_train[k], acts_dict_test[k])
    acts_dict_train_test['obs'] = (obs_train, obs_test)

    return acts_dict_train_test

def validate_model(model, val_acts, val_values, binary):
    outputs = model(val_acts)
    score = binary_accuracy(outputs, val_values) if binary else r2(outputs, val_values)
    return score

In [17]:
def train_model(train_acts, train_values, input_dim, l1_lambda, lr, epochs, binary):
    if binary:
        model = nn.Sequential(nn.Linear(input_dim, 1), nn.Sigmoid()).to(device)
    else:
        model = nn.Sequential(nn.Linear(input_dim, 1)).to(device)
        
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(train_acts)
        train_loss = loss_fn(outputs, train_values)
        l1_reg = torch.tensor(0.0, device=device)
        for param in model.parameters():
            l1_reg += torch.norm(param, p=1)
        train_loss += l1_lambda * l1_reg
        train_loss.backward()
        optimizer.step()
    return model

def cross_validate(activations, values_train, l1_lambdas, kfold, lr, epochs, binary):
    best_l1_lambda = 0
    best_avg_score = -np.inf
    for l1_lambda in l1_lambdas:
        scores = []
        for train_idx, val_idx in kfold.split(activations[0]):
            train_acts = activations[0][train_idx].reshape(len(train_idx), -1).detach()
            val_acts = activations[0][val_idx].reshape(len(val_idx), -1).detach()
            train_values = torch.tensor(values_train[train_idx]).reshape(-1, 1).float().to(device)
            val_values = torch.tensor(values_train[val_idx]).reshape(-1, 1).float().to(device)
            model = train_model(train_acts, train_values, train_acts.shape[1], l1_lambda, lr, epochs, binary)
            score = validate_model(model, val_acts, val_values, binary)
            scores.append(score)
        avg_score = np.mean(scores)
        if avg_score > best_avg_score:
            best_avg_score = avg_score
            best_l1_lambda = l1_lambda
    print(f"Best l1_lambda: {best_l1_lambda} with avg score: {best_avg_score:.3f}")
    return best_l1_lambda

def final_training_and_testing(concept_instances, kfold, lr, epochs, l1_lambdas):
    for concept in concept_instances.values():
        acts_dict_train_test = prepare_data(concept)
        train_values = torch.tensor(concept.values_train).reshape(-1, 1).float().to(device)
        test_values = torch.tensor(concept.values_test).reshape(-1, 1).float().to(device)
        print(f"Concept: {concept.name}")
        if concept.binary:
            continue
        for k, activations in acts_dict_train_test.items():
            best_l1_lambda = cross_validate(activations, concept.values_train, l1_lambdas, kfold, lr, epochs, concept.binary)
            train_acts = activations[0].reshape(activations[0].shape[0], -1).detach()
            test_acts = activations[1].reshape(activations[1].shape[0], -1).detach()
            final_model = train_model(train_acts, train_values, train_acts.shape[1], best_l1_lambda, lr, epochs, concept.binary)
            final_score = validate_model(final_model, test_acts, test_values, concept.binary)
            print(f"Final Test Score layer {k} with l1_lambda={best_l1_lambda}: {final_score:.3f}")
            reg = linear_model.LassoCV(max_iter=50, cv=5, alphas=l1_lambdas)
            reg.fit(train_acts.cpu().numpy(), train_values.cpu().numpy())
            pred = reg.predict(test_acts.cpu().numpy())
            score = r2_score(test_values.cpu().numpy(), pred)
            print(f"LassoCV Final Test Score layer {k} with l1_lambda={reg.alpha_}: {score:.3f}")

loss_fn = nn.MSELoss()
n_splits = 20
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=0)
lr = 0.001
epochs = 25
l1_lambdas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
final_training_and_testing(concept_instances, kfold, lr, epochs, l1_lambdas)

Concept: random (b)
Concept: all lives (b)
Concept: last life (b)
Concept: reward (b)
Concept: ball collision (b)
Concept: ball low (b)
Concept: ball left paddle (b)
Concept: ball right paddle (b)
Concept: ball same x paddle (b)
Concept: ball distance paddle
Best l1_lambda: 0.001 with avg score: 0.007
Final Test Score layer 0 with l1_lambda=0.001: 0.000


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 0 with l1_lambda=0.001: 0.467
Best l1_lambda: 0.001 with avg score: 0.114
Final Test Score layer 1 with l1_lambda=0.001: 0.145


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 1 with l1_lambda=0.001: 0.451
Best l1_lambda: 0.001 with avg score: 0.262
Final Test Score layer 2 with l1_lambda=0.001: 0.071


...

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 0 with l1_lambda=0.001: 0.630
Best l1_lambda: 0.1 with avg score: 0.039
Final Test Score layer 1 with l1_lambda=0.1: 0.000


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 1 with l1_lambda=0.001: 0.501


KeyboardInterrupt: 

In [ ]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for concept in concept_instances.values():
    print(f"Concept: {concept.name}")
    acts_dict_train_test = prepare_data(concept)

    for k, activations in acts_dict_train_test.items():
        optimal_l1_lambda = 0
        best_avg_score = -np.inf
        
        # Find the best l1_lambda using k-fold CV
        for l1_lambda in np.linspace(0.01, 1, num=10):  # Set your range and number of l1_lambda values to test
            scores = []

            for train_idx, val_idx in kfold.split(activations[0]):
                train_acts = activations[0][train_idx].reshape(len(train_idx), -1).detach()
                val_acts = activations[0][val_idx].reshape(len(val_idx), -1).detach()

                train_values = torch.tensor(concept.values_train[train_idx]).reshape(-1, 1).float().to(device)
                val_values = torch.tensor(concept.values_train[val_idx]).reshape(-1, 1).float().to(device)

                # Model and optimizer initialization
                model = nn.Sequential(nn.Linear(train_acts.shape[1], 1), nn.Sigmoid() if concept.binary else nn.Identity()).to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training loop
                for epoch in range(epochs):
                    optimizer.zero_grad()
                    outputs = model(train_acts)
                    train_loss = loss_fn(outputs, train_values)

                    # L1 regularization
                    l1_reg = torch.tensor(0.0, device=device)
                    for param in model.parameters():
                        l1_reg += torch.norm(param, p=1)
                    train_loss += l1_lambda * l1_reg
                    
                    train_loss.backward()
                    optimizer.step()

                # Validation
                outputs = model(val_acts)
                score = binary_accuracy(outputs, val_values) if concept.binary else r2(outputs, val_values)
                scores.append(score)

            avg_score = np.mean(scores)
            if avg_score > best_avg_score:
                best_avg_score = avg_score
                optimal_l1_lambda = l1_lambda

            # print(f"Average validation score with l1_lambda={l1_lambda:.2f} for layer {k}: {avg_score:.3f}")

        print(f"Optimal l1_lambda for layer {k}: {optimal_l1_lambda:.2f} with average validation score: {best_avg_score:.3f}")
        
        # Using the best l1_lambda, train on the entire training set and evaluate on the test set
        train_acts, test_acts = activations
        train_acts = train_acts.reshape(train_acts.shape[0], -1).detach()
        test_acts = test_acts.reshape(test_acts.shape[0], -1).detach()

        train_values = torch.tensor(concept.values_train).reshape(-1, 1).float().to(device)
        test_values = torch.tensor(concept.values_test).reshape(-1, 1).float().to(device)

        model = nn.Sequential(nn.Linear(train_acts.shape[1], 1), nn.Sigmoid() if concept.binary else nn.Identity()).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Training loop with best l1_lambda
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = model(train_acts)
            train_loss = loss_fn(outputs, train_values)

            # L1 regularization
            l1_reg = torch.tensor(0.0, device=device)
            for param in model.parameters():
                l1_reg += torch.norm(param, p=1)
            train_loss += optimal_l1_lambda * l1_reg
            
            train_loss.backward()
            optimizer.step()

        # Test the model on the test dataset
        outputs = model(test_acts)
        if concept.binary:
            print(f"Test accuracy layer {k} with l1_lambda={optimal_l1_lambda:.2f}: {binary_accuracy(outputs, test_values):.3f}")
        else:
            print(f"Test R2 layer {k} with l1_lambda={optimal_l1_lambda:.2f}: {r2(outputs, test_values):.3f}")

Concept: random (b)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [18]:
batch_size = 1000
learning_rate = 0.001
epochs = 100
l1_lambda = 0.0001
for concept in concept_instances.values():
    print(f"Concept: {concept.name}")
    if concept.binary:
        continue
    acts_dict_train_test = prepare_data(concept)
    # train probe for each layer
    for k, activations in acts_dict_train_test.items():
        train_acts, test_acts = activations
        train_acts = train_acts.reshape(train_acts.shape[0], -1).detach()
        test_acts = test_acts.reshape(test_acts.shape[0], -1).detach()

        train_values = torch.tensor(concept.values_train).reshape(-1, 1).float().to(device)
        test_values = torch.tensor(concept.values_test).reshape(-1, 1).float().to(device)

        # create 1 layer network as logistic regression
        if concept.binary:
            model = nn.Sequential(nn.Linear(train_acts.shape[1], 1), nn.Sigmoid()).to(device)
        else:
            model = nn.Sequential(nn.Linear(train_acts.shape[1], 1)).to(device)

        # train the model
        for epoch in range(epochs):
            optimizer = optim.Adam(model.parameters())
            total_train_loss = 0.0

            for batch_idx in range(len(train_acts) // batch_size):
                start_idx = batch_idx * batch_size
                end_idx = start_idx + batch_size

                optimizer.zero_grad()
                batch_train_acts = train_acts[start_idx:end_idx]
                batch_train_values = train_values[start_idx:end_idx]
                outputs = model(batch_train_acts)
                train_loss = loss_fn(outputs, batch_train_values)

                # Calculate L1 regularization for weights only
                l1_reg = torch.tensor(0.0, device=device)
                for name, param in model.named_parameters():
                    if 'weight' in name:  # check if 'param' is weight
                        l1_reg += torch.norm(param, p=1)

                # Add L1 regularization to the loss
                train_loss += l1_lambda * l1_reg

                train_loss.backward()
                optimizer.step()
                total_train_loss += train_loss.item()

        # test the model
        outputs = model(test_acts)
        if concept.binary:
            print(f"Test accuracy layer {k}: {binary_accuracy(outputs, test_values):.3f}")
        else:
            print(f"Test R2 layer {k}: {r2(outputs, test_values):.3f}")

        reg = linear_model.LassoCV(max_iter=50, cv=5, alphas=l1_lambdas)
        reg.fit(train_acts.cpu().numpy(), train_values.cpu().numpy())
        pred = reg.predict(test_acts.cpu().numpy())
        score = r2_score(test_values.cpu().numpy(), pred)
        print(f"LassoCV Final Test Score layer {k} with l1_lambda={reg.alpha_}: {score:.3f}")

Concept: random (b)
Concept: all lives (b)
Concept: last life (b)
Concept: reward (b)
Concept: ball collision (b)
Concept: ball low (b)
Concept: ball left paddle (b)
Concept: ball right paddle (b)
Concept: ball same x paddle (b)
Concept: ball distance paddle
Test R2 layer 0: 0.000


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 0 with l1_lambda=0.001: 0.467
Test R2 layer 1: 0.000


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 1 with l1_lambda=0.001: 0.451
Test R2 layer 2: 0.000


...

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 5 with l1_lambda=0.001: 0.988
Test R2 layer 7: 0.000
LassoCV Final Test Score layer 7 with l1_lambda=0.001: 0.930
Test R2 layer obs: 0.000


/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer obs with l1_lambda=0.001: 0.991
Concept: x diff


KeyboardInterrupt: 

In [38]:
concept = concept_instances['ball x']
batch_size = 64
l1_lambda = 0.0
print(f"Concept: {concept.name}")
acts_dict_train_test = prepare_data(concept)
# train probe for each layer
for k, activations in acts_dict_train_test.items():
    train_acts, test_acts = activations
    train_acts = train_acts.reshape(train_acts.shape[0], -1).detach()
    test_acts = test_acts.reshape(test_acts.shape[0], -1).detach()

    train_values = torch.tensor(concept.values_train).reshape(-1, 1).float().to(device)
    test_values = torch.tensor(concept.values_test).reshape(-1, 1).float().to(device)

    # create 1 layer network as logistic regression
    if concept.binary:
        model = nn.Sequential(nn.Linear(train_acts.shape[1], 1), nn.Sigmoid()).to(device)
    else:
        model = nn.Sequential(nn.Linear(train_acts.shape[1], 1)).to(device)

    # train the model
    for epoch in range(epochs):
        optimizer = optim.Adam(model.parameters())
        total_train_loss = 0.0

        for batch_idx in range(len(train_acts) // batch_size):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            optimizer.zero_grad()
            batch_train_acts = train_acts[start_idx:end_idx]
            batch_train_values = train_values[start_idx:end_idx]
            outputs = model(batch_train_acts)
            train_loss = loss_fn(outputs, batch_train_values)

            # Calculate L1 regularization for weights only
            l1_reg = torch.tensor(0.0, device=device)
            for name, param in model.named_parameters():
                if 'weight' in name:  # check if 'param' is weight
                    l1_reg += torch.norm(param, p=1)

            # Add L1 regularization to the loss
            train_loss += l1_lambda * l1_reg
            print(train_loss, l1_lambda * l1_reg)
            train_loss.backward()
            optimizer.step()
            total_train_loss += train_loss.item()

    # test the model
    outputs = model(test_acts)
    if concept.binary:
        print(f"Test accuracy layer {k}: {binary_accuracy(outputs, test_values):.3f}")
    else:
        print(f"Test R2 layer {k}: {r2(outputs, test_values):.3f}")

    reg = linear_model.LassoCV(max_iter=50, cv=5, alphas=l1_lambdas)
    reg.fit(train_acts.cpu().numpy(), train_values.cpu().numpy())
    pred = reg.predict(test_acts.cpu().numpy())
    score = r2_score(test_values.cpu().numpy(), pred)
    print(f"LassoCV Final Test Score layer {k} with l1_lambda={reg.alpha_}: {score:.3f}")
    break

Concept: ball x
tensor(2.3632, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(51.0095, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(3.8475, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(11.0034, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(28.3349, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(18.2473, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(3.0446, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(2.1192, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(11.1805, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/.conda/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV Final Test Score layer 0 with l1_lambda=0.001: 0.688


: 

In [25]:
import matplotlib.pyplot as plt

In [35]:
reg.coef_.mean(), reg.coef_.std()

(-0.00020997308, 0.0533668)

In [34]:
# go through parameters in model
for name, param in model.named_parameters():
    # if bias
    if 'bias' in name:
        print(name, param)
    else:
        print(name, param.mean(), param.std())

0.weight tensor(-1.3501e-05, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0051, device='cuda:0', grad_fn=<StdBackward0>)
0.bias Parameter containing:
tensor([-0.0025], device='cuda:0', requires_grad=True)
